In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics.pairwise import cosine_similarity
import joblib

In [10]:

# Load datasets once
df1 = pd.read_csv("data/dataset1.csv")  # Ensure no NaN values
df2 = pd.read_csv("data/dataset2.csv")

# Extract invoice numbers
invoices1 = df1["Data Set 1"].astype(str).tolist()
invoices2 = df2["Data Set 2"].astype(str).tolist()


In [11]:
len(invoices1)

11573

In [12]:
type(invoices1)

list

In [13]:

# Create TF-IDF Vectorizer (train on both datasets)
vectorizer = TfidfVectorizer(analyzer="char_wb", ngram_range=(2, 4))
vectorizer.fit(invoices1 + invoices2)


TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 4))

In [14]:

# Transform invoices
tfidf_matrix1 = vectorizer.transform(invoices1)
tfidf_matrix2 = vectorizer.transform(invoices2)


### Giving MemoryError


In [15]:

# # Compute similarity
# similarity_matrix = cosine_similarity(tfidf_matrix1, tfidf_matrix2)


### NewNew


### Alternate approach


In [16]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def batch_cosine_similarity(matrix1, matrix2, batch_size=1000):
    num_rows = matrix1.shape[0]
    similarity_matrix = np.zeros((matrix1.shape[0], matrix2.shape[0]))
    
    for i in range(0, num_rows, batch_size):
        end = min(i + batch_size, num_rows)
        similarity_matrix[i:end] = cosine_similarity(
            matrix1[i:end], 
            matrix2
        )
    
    return similarity_matrix

In [17]:
import numpy as np
import pandas as pd

similarity_matrix = batch_cosine_similarity(tfidf_matrix1, tfidf_matrix2)
threshold = np.mean(similarity_matrix) + np.std(similarity_matrix)
matches = []

# Create masks for tracking matched invoices
matched_invoice2 = set()

for i, row in enumerate(similarity_matrix):
    best_match_idx = np.argmax(row)
    best_match_score = row[best_match_idx]
    
    if best_match_score >= threshold and best_match_idx not in matched_invoice2:
        matches.append((invoices1[i], invoices2[best_match_idx], best_match_score))
        matched_invoice2.add(best_match_idx)

matches_df = pd.DataFrame(matches, columns=["Invoice1", "Matched_Invoice2", "Similarity_Score"])

try:
    matches_df.to_csv("matched_invoices.csv", index=False)
except IOError as e:
    print(f"Error saving matches to CSV: {e}")

In [18]:

# **Feedback Handling & Model Retraining**
try:
    feedback_df = pd.read_csv("feedback.csv")  # Load feedback if it exists
except FileNotFoundError:
    feedback_df = pd.DataFrame(columns=["invoice1", "invoice2", "similarity_score", "label"])


In [19]:
feedback_df

Empty DataFrame
Columns: [invoice1, invoice2, similarity_score, label]
Index: []

In [20]:
# Validate feedback_df columns
required_columns = ["invoice1", "invoice2"]
if not all(col in feedback_df.columns for col in required_columns):
    raise ValueError(f"feedback_df missing required columns: {required_columns}")

# Merge with validation
train_df = pd.DataFrame(matches, columns=["invoice1", "invoice2", "similarity_score"])
merged_df = train_df.merge(feedback_df, on=["invoice1", "invoice2"], how="left")


In [21]:
merged_df

invoice1                   invoice2  similarity_score_x  \
0         5514104810                 5514104892            0.668748   
1         5719101009                 5719101588            0.624549   
2         5514105044                 5514105198            0.587657   
3        B2064/23-24                 210 /23-24            0.264735   
4        B2855/23-24  SALES B - Bill # 000B2855            0.276246   
...              ...                        ...                 ...   
9359    KAA-2223/332               KAA/2223/332            0.700428   
9360     SI2223/4067            KIN/SI2223/4067            0.653983   
9361   SPB-0049-0423              SPB 0049-0423            0.739689   
9362  TAC-23-24-0002             TAC/23-24/0002            0.415166   
9363      TPP-03-014                 TPP_03_014            0.270582   

     similarity_score_y label  
0                   NaN   NaN  
1                   NaN   NaN  
2                   NaN   NaN  
3                   NaN   NaN  
4                   NaN   NaN  
...                 ...   ...  
9359                NaN   NaN  
9360                NaN   NaN  
9361                NaN   NaN  
9362                NaN   NaN  
9363                NaN   NaN  

[9364 rows x 5 columns]

In [22]:

try:
    merged_df.to_csv("mergedf.csv", index=False)
except IOError as e:
    print(f"Error saving matches to CSV: {e}")

In [23]:
merged_df.shape

(9364, 5)

### Cleaning


In [24]:
# Check merge inputs
print("Shape before merge:", train_df.shape)
print("Shape of feedback_df:", feedback_df.shape)
print("\nSample of unmatched records:")
merged_df = train_df.merge(feedback_df, on=["invoice1", "invoice2"], how="left")
unmatched = merged_df[merged_df["label"].isna()]
print(unmatched.head())

Shape before merge: (9364, 3)
Shape of feedback_df: (0, 4)

Sample of unmatched records:
      invoice1                   invoice2  similarity_score_x  \
0   5514104810                 5514104892            0.668748   
1   5719101009                 5719101588            0.624549   
2   5514105044                 5514105198            0.587657   
3  B2064/23-24                 210 /23-24            0.264735   
4  B2855/23-24  SALES B - Bill # 000B2855            0.276246   

  similarity_score_y label  
0                NaN   NaN  
1                NaN   NaN  
2                NaN   NaN  
3                NaN   NaN  
4                NaN   NaN  


In [25]:
# Data cleaning before merge
def clean_and_merge():
    # Standardize invoice IDs
    train_df["invoice1"] = train_df["invoice1"].astype(str).str.strip()
    train_df["invoice2"] = train_df["invoice2"].astype(str).str.strip()
    feedback_df["invoice1"] = feedback_df["invoice1"].astype(str).str.strip()
    feedback_df["invoice2"] = feedback_df["invoice2"].astype(str).str.strip()
    
    # Check for mismatched keys
    train_keys = set(zip(train_df["invoice1"], train_df["invoice2"]))
    feedback_keys = set(zip(feedback_df["invoice1"], feedback_df["invoice2"]))
    missing_keys = train_keys - feedback_keys
    
    if missing_keys:
        print(f"Found {len(missing_keys)} invoice pairs without feedback")
        print("Sample missing pairs:", list(missing_keys)[:5])
    
    # Perform merge
    return train_df.merge(feedback_df, on=["invoice1", "invoice2"], how="left")

merged_df = clean_and_merge()

Found 9364 invoice pairs without feedback
Sample missing pairs: [('1455-2207838INSC-(22-23)', '2207838'), ('1129000095', '1129000095'), ('1459-2224881INSC-(22-23)', '2224881'), ('BBF23326508', 'BBF23326508'), ('14350', '21-22/14350')]


In [26]:
print("Train columns:", train_df.columns.tolist())
print("Feedback columns:", feedback_df.columns.tolist())

Train columns: ['invoice1', 'invoice2', 'similarity_score']
Feedback columns: ['invoice1', 'invoice2', 'similarity_score', 'label']


In [27]:
train_df

invoice1                   invoice2  similarity_score
0         5514104810                 5514104892          0.668748
1         5719101009                 5719101588          0.624549
2         5514105044                 5514105198          0.587657
3        B2064/23-24                 210 /23-24          0.264735
4        B2855/23-24  SALES B - Bill # 000B2855          0.276246
...              ...                        ...               ...
9359    KAA-2223/332               KAA/2223/332          0.700428
9360     SI2223/4067            KIN/SI2223/4067          0.653983
9361   SPB-0049-0423              SPB 0049-0423          0.739689
9362  TAC-23-24-0002             TAC/23-24/0002          0.415166
9363      TPP-03-014                 TPP_03_014          0.270582

[9364 rows x 3 columns]

In [28]:
merged_df = train_df.merge(
    feedback_df[["invoice1", "invoice2", "label"]],  # Only select needed columns
    on=["invoice1", "invoice2"],
    how="left"
)

In [29]:
merged_df

invoice1                   invoice2  similarity_score label
0         5514104810                 5514104892          0.668748   NaN
1         5719101009                 5719101588          0.624549   NaN
2         5514105044                 5514105198          0.587657   NaN
3        B2064/23-24                 210 /23-24          0.264735   NaN
4        B2855/23-24  SALES B - Bill # 000B2855          0.276246   NaN
...              ...                        ...               ...   ...
9359    KAA-2223/332               KAA/2223/332          0.700428   NaN
9360     SI2223/4067            KIN/SI2223/4067          0.653983   NaN
9361   SPB-0049-0423              SPB 0049-0423          0.739689   NaN
9362  TAC-23-24-0002             TAC/23-24/0002          0.415166   NaN
9363      TPP-03-014                 TPP_03_014          0.270582   NaN

[9364 rows x 4 columns]

### break


In [30]:
train_df

invoice1                   invoice2  similarity_score
0         5514104810                 5514104892          0.668748
1         5719101009                 5719101588          0.624549
2         5514105044                 5514105198          0.587657
3        B2064/23-24                 210 /23-24          0.264735
4        B2855/23-24  SALES B - Bill # 000B2855          0.276246
...              ...                        ...               ...
9359    KAA-2223/332               KAA/2223/332          0.700428
9360     SI2223/4067            KIN/SI2223/4067          0.653983
9361   SPB-0049-0423              SPB 0049-0423          0.739689
9362  TAC-23-24-0002             TAC/23-24/0002          0.415166
9363      TPP-03-014                 TPP_03_014          0.270582

[9364 rows x 3 columns]

In [31]:
# Define labels
UNMATCHED = 0
PARTIAL_MATCH = 1
EXACT_MATCH = 2

# Create label based on similarity score thresholds
def assign_label(similarity_score):
    if similarity_score <= 0.034:
        return UNMATCHED
    elif similarity_score >= 0.95:
        return EXACT_MATCH
    else:
        return PARTIAL_MATCH

# Apply labeling logic
merged_df["label"] = merged_df["similarity_score"].apply(assign_label)

# Feature Engineering
X_train = vectorizer.transform(merged_df["invoice1"] + " | " + merged_df["invoice2"])
y_train = merged_df["label"].astype(int)

# Print distribution of labels
print("Label distribution:")
print(merged_df["label"].value_counts().sort_index())

Label distribution:
label
1    5593
2    3771
Name: count, dtype: int64


In [32]:
y_train

0       1
1       1
2       1
3       1
4       1
       ..
9359    1
9360    1
9361    1
9362    1
9363    1
Name: label, Length: 9364, dtype: int32

In [33]:

# # Train the model incrementally with new data
# try:
#     model.partial_fit(X_train, y_train, classes=np.array([0, 1, 2]))
# except ValueError as e:
#     # If model was previously trained for binary classification, create new model
#     model = PassiveAggressiveClassifier(max_iter=1000, random_state=42)
#     model.partial_fit(X_train, y_train, classes=np.array([0, 1, 2]))
# # Train the model incrementally with new data
# model.partial_fit(X_train, y_train, classes=np.array([0, 1, 2]))
# # Validate model and vectorizer before saving
# if hasattr(model, 'classes_') and hasattr(vectorizer, 'vocabulary_'):
#     try:
#         joblib.dump(model, "invoice_match_model.pkl")
#         joblib.dump(vectorizer, "vectorizer.pkl")
#     except Exception as e:
#         print(f"Error saving model: {e}")
# else:
#     raise ValueError("Model or vectorizer not properly fitted")

# Initialize model if not exists
if 'model' not in locals():
    model = PassiveAggressiveClassifier(max_iter=1000, random_state=42)

# Train model once
try:
    model.partial_fit(X_train, y_train, classes=np.array([0, 1, 2]))
except ValueError as e:
    print(f"Training error: {e}")
    # Verify input shapes
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"Unique labels: {np.unique(y_train)}")
    raise

# Validate and save
if hasattr(model, 'classes_') and hasattr(vectorizer, 'vocabulary_'):
    joblib.dump(model, "invoice_match_model.joblib")
    joblib.dump(vectorizer, "vectorizer.joblib")
else:
    raise ValueError("Model or vectorizer not properly fitted")

In [34]:
test_df = pd.read_csv("data/exact_match_dataset.csv")

test_df

Match ID  \
{'Data Set 1': '5514104810'  'Data Set 2': 'TX1718301226'  'predicted_label': 2  'similarity_score': 0.0} 'Data Set 2': 'TX1718301226' 'predicted_label': 2   'similarity_score': 0.0}   
2                           OMPL/003/2018-19              3                     Partial Matched           NaN                          NaN                                         NaN   
3                           JP/222300006                  6                     Partial Matched           NaN                          NaN                                         NaN   
4                           JP/222300007                  7                     Partial Matched           NaN                          NaN                                         NaN   
5                           JP/222300008                  8                     Partial Matched           NaN                          NaN                                         NaN   
...                                                                                                                                                                                ...   
11629                       24303BO1123                   NaN                   Unmatched                 NaN                          NaN                                         NaN   
11630                       24303BO963                    NaN                   Unmatched                 NaN                          NaN                                         NaN   
11631                       24303BO369                    NaN                   Unmatched                 NaN                          NaN                                         NaN   
11632                       24303BO967                    NaN                   Unmatched                 NaN                          NaN                                         NaN   
11633                       24303BO1044                   NaN                   Unmatched                 NaN                          NaN                                         NaN   

                                                                                                                                                               Data Set 1  \
{'Data Set 1': '5514104810'  'Data Set 2': 'TX1718301226'  'predicted_label': 2  'similarity_score': 0.0} 'Data Set 2': 'TX1718301226' 'predicted_label': 2  JP/232400001   
2                           OMPL/003/2018-19              3                     Partial Matched           NaN                          NaN                            NaN   
3                           JP/222300006                  6                     Partial Matched           NaN                          NaN                            NaN   
4                           JP/222300007                  7                     Partial Matched           NaN                          NaN                            NaN   
5                           JP/222300008                  8                     Partial Matched           NaN                          NaN                            NaN   
...                                                                                                                                                                   ...   
11629                       24303BO1123                   NaN                   Unmatched                 NaN                          NaN                            NaN   
11630                       24303BO963                    NaN                   Unmatched                 NaN                          NaN                            NaN   
11631                       24303BO369                    NaN                   Unmatched                 NaN                          NaN                            NaN   
11632                       24303BO967                    NaN                   Unmatched                 NaN                          NaN                            NaN   
11633                       24303BO104

In [35]:
# Define labels
UNMATCHED = 0
PARTIAL_MATCH = 1
EXACT_MATCH = 2

# Create label based on similarity score thresholds
def assign_label(status):
    if status == "Unmatched":
        return UNMATCHED
    elif status == "Exact Matched":
        return EXACT_MATCH
    else:
        return PARTIAL_MATCH

# Apply labeling logic
test_df["label"] = test_df["Status"].apply(assign_label)

In [36]:
test_df

Match ID  \
{'Data Set 1': '5514104810'  'Data Set 2': 'TX1718301226'  'predicted_label': 2  'similarity_score': 0.0} 'Data Set 2': 'TX1718301226' 'predicted_label': 2   'similarity_score': 0.0}   
2                           OMPL/003/2018-19              3                     Partial Matched           NaN                          NaN                                         NaN   
3                           JP/222300006                  6                     Partial Matched           NaN                          NaN                                         NaN   
4                           JP/222300007                  7                     Partial Matched           NaN                          NaN                                         NaN   
5                           JP/222300008                  8                     Partial Matched           NaN                          NaN                                         NaN   
...                                                                                                                                                                                ...   
11629                       24303BO1123                   NaN                   Unmatched                 NaN                          NaN                                         NaN   
11630                       24303BO963                    NaN                   Unmatched                 NaN                          NaN                                         NaN   
11631                       24303BO369                    NaN                   Unmatched                 NaN                          NaN                                         NaN   
11632                       24303BO967                    NaN                   Unmatched                 NaN                          NaN                                         NaN   
11633                       24303BO1044                   NaN                   Unmatched                 NaN                          NaN                                         NaN   

                                                                                                                                                               Data Set 1  \
{'Data Set 1': '5514104810'  'Data Set 2': 'TX1718301226'  'predicted_label': 2  'similarity_score': 0.0} 'Data Set 2': 'TX1718301226' 'predicted_label': 2  JP/232400001   
2                           OMPL/003/2018-19              3                     Partial Matched           NaN                          NaN                            NaN   
3                           JP/222300006                  6                     Partial Matched           NaN                          NaN                            NaN   
4                           JP/222300007                  7                     Partial Matched           NaN                          NaN                            NaN   
5                           JP/222300008                  8                     Partial Matched           NaN                          NaN                            NaN   
...                                                                                                                                                                   ...   
11629                       24303BO1123                   NaN                   Unmatched                 NaN                          NaN                            NaN   
11630                       24303BO963                    NaN                   Unmatched                 NaN                          NaN                            NaN   
11631                       24303BO369                    NaN                   Unmatched                 NaN                          NaN                            NaN   
11632                       24303BO967                    NaN                   Unmatched                 NaN                          NaN                            NaN   
11633                       24303BO104

In [37]:
# import pandas as pd
# import numpy as np
# from sklearn.metrics import classification_report, confusion_matrix

# def test_model_performance(model, test_data, threshold=0.5):
#     """
#     Test the invoice matching model performance
#     """
#     # Prepare test data
#     X_test = test_data[['similarity_score']]
#     y_test = test_data['label']
    
#     # Get predictions
#     y_pred_proba = model.predict_proba(X_test)[:, 1]
#     y_pred = (y_pred_proba >= threshold).astype(int)
    
#     # Calculate metrics
#     print("Classification Report:")
#     print(classification_report(y_test, y_pred))
#     print("\nConfusion Matrix:")
#     print(confusion_matrix(y_test, y_pred))
    
#     return y_pred, y_pred_proba

# def cross_validate_thresholds(model, validation_data, thresholds=[0.3, 0.4, 0.5, 0.6, 0.7]):
#     """
#     Test different threshold values
#     """
#     results = []
#     X_val = validation_data[['similarity_score']]
#     y_val = validation_data['label']
    
#     for threshold in thresholds:
#         y_pred_proba = model.predict_proba(X_val)[:, 1]
#         y_pred = (y_pred_proba >= threshold).astype(int)
        
#         tp = np.sum((y_pred == 1) & (y_val == 1))
#         fp = np.sum((y_pred == 1) & (y_val == 0))
#         precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        
#         results.append({
#             'threshold': threshold,
#             'precision': precision,
#             'true_matches': np.sum(y_pred == 1)
#         })
    
#     return pd.DataFrame(results)

# # Usage example:
# # if __name__ == "__main__":
#     # Load your trained model and test data
# test_data = pd.read_csv('test_data.csv')

# # Test performance
# y_pred, y_pred_proba = test_model_performance(model, test_data)

# # Find optimal threshold
# threshold_results = cross_validate_thresholds(model, test_data)
# print("\nThreshold Analysis:")
# print(threshold_results)

In [38]:
test_df

Match ID  \
{'Data Set 1': '5514104810'  'Data Set 2': 'TX1718301226'  'predicted_label': 2  'similarity_score': 0.0} 'Data Set 2': 'TX1718301226' 'predicted_label': 2   'similarity_score': 0.0}   
2                           OMPL/003/2018-19              3                     Partial Matched           NaN                          NaN                                         NaN   
3                           JP/222300006                  6                     Partial Matched           NaN                          NaN                                         NaN   
4                           JP/222300007                  7                     Partial Matched           NaN                          NaN                                         NaN   
5                           JP/222300008                  8                     Partial Matched           NaN                          NaN                                         NaN   
...                                                                                                                                                                                ...   
11629                       24303BO1123                   NaN                   Unmatched                 NaN                          NaN                                         NaN   
11630                       24303BO963                    NaN                   Unmatched                 NaN                          NaN                                         NaN   
11631                       24303BO369                    NaN                   Unmatched                 NaN                          NaN                                         NaN   
11632                       24303BO967                    NaN                   Unmatched                 NaN                          NaN                                         NaN   
11633                       24303BO1044                   NaN                   Unmatched                 NaN                          NaN                                         NaN   

                                                                                                                                                               Data Set 1  \
{'Data Set 1': '5514104810'  'Data Set 2': 'TX1718301226'  'predicted_label': 2  'similarity_score': 0.0} 'Data Set 2': 'TX1718301226' 'predicted_label': 2  JP/232400001   
2                           OMPL/003/2018-19              3                     Partial Matched           NaN                          NaN                            NaN   
3                           JP/222300006                  6                     Partial Matched           NaN                          NaN                            NaN   
4                           JP/222300007                  7                     Partial Matched           NaN                          NaN                            NaN   
5                           JP/222300008                  8                     Partial Matched           NaN                          NaN                            NaN   
...                                                                                                                                                                   ...   
11629                       24303BO1123                   NaN                   Unmatched                 NaN                          NaN                            NaN   
11630                       24303BO963                    NaN                   Unmatched                 NaN                          NaN                            NaN   
11631                       24303BO369                    NaN                   Unmatched                 NaN                          NaN                            NaN   
11632                       24303BO967                    NaN                   Unmatched                 NaN                          NaN                            NaN   
11633                       24303BO104

In [39]:
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

# Handle NaN values in test data
test_df_clean = test_df.copy()
test_df_clean["Data Set 1"] = test_df_clean["Data Set 1"].fillna("")
test_df_clean["Data Set 2"] = test_df_clean["Data Set 2"].fillna("")
test_df_clean["label"] = test_df_clean["label"].fillna(-1)  # Mark missing labels

# Filter out rows with missing labels
valid_mask = test_df_clean["label"] != -1
test_df_valid = test_df_clean[valid_mask]


In [40]:
test_df_valid

Match ID  \
{'Data Set 1': '5514104810'  'Data Set 2': 'TX1718301226'  'predicted_label': 2  'similarity_score': 0.0} 'Data Set 2': 'TX1718301226' 'predicted_label': 2   'similarity_score': 0.0}   
2                           OMPL/003/2018-19              3                     Partial Matched           NaN                          NaN                                         NaN   
3                           JP/222300006                  6                     Partial Matched           NaN                          NaN                                         NaN   
4                           JP/222300007                  7                     Partial Matched           NaN                          NaN                                         NaN   
5                           JP/222300008                  8                     Partial Matched           NaN                          NaN                                         NaN   
...                                                                                                                                                                                ...   
11629                       24303BO1123                   NaN                   Unmatched                 NaN                          NaN                                         NaN   
11630                       24303BO963                    NaN                   Unmatched                 NaN                          NaN                                         NaN   
11631                       24303BO369                    NaN                   Unmatched                 NaN                          NaN                                         NaN   
11632                       24303BO967                    NaN                   Unmatched                 NaN                          NaN                                         NaN   
11633                       24303BO1044                   NaN                   Unmatched                 NaN                          NaN                                         NaN   

                                                                                                                                                               Data Set 1  \
{'Data Set 1': '5514104810'  'Data Set 2': 'TX1718301226'  'predicted_label': 2  'similarity_score': 0.0} 'Data Set 2': 'TX1718301226' 'predicted_label': 2  JP/232400001   
2                           OMPL/003/2018-19              3                     Partial Matched           NaN                          NaN                                  
3                           JP/222300006                  6                     Partial Matched           NaN                          NaN                                  
4                           JP/222300007                  7                     Partial Matched           NaN                          NaN                                  
5                           JP/222300008                  8                     Partial Matched           NaN                          NaN                                  
...                                                                                                                                                                   ...   
11629                       24303BO1123                   NaN                   Unmatched                 NaN                          NaN                                  
11630                       24303BO963                    NaN                   Unmatched                 NaN                          NaN                                  
11631                       24303BO369                    NaN                   Unmatched                 NaN                          NaN                                  
11632                       24303BO967                    NaN                   Unmatched                 NaN                          NaN                                  
11633                       24303BO104

In [41]:

# Validate model performance
X_test = vectorizer.transform(test_df_valid["Data Set 1"].astype(str) + " | " + test_df_valid["Data Set 2"].astype(str))
y_pred = model.predict(X_test)

In [42]:
y_pred

array([1, 2, 2, ..., 2, 2, 2])

In [43]:
# Convert model predictions to labels
test_df["y_pred"] = y_pred

# If you need to map predictions to specific labels, use:
def map_predictions(pred):
    label_map = {
        0: "Unmatched",
        1: "Partial Match", 
        2: "Exact Match"
    }
    return label_map.get(pred, "Unknown")

# Apply mapping if needed
test_df["y_pred_label"] = test_df["y_pred"].map(map_predictions)

In [44]:
import joblib
# Generate classification report for valid predictions only
print(classification_report(test_df_valid["label"], y_pred))

# Calculate accuracy on valid data
accuracy = (y_pred == test_df_valid["label"]).mean()
print(f"Number of valid test samples: {len(test_df_valid)}")
print(f"Number of samples with missing data: {len(test_df) - len(test_df_valid)}")

if accuracy >= 0.80:
    joblib.dump(model, "invoice_match_model.joblib")
    joblib.dump(vectorizer, "vectorizer.joblib")
else:
    print(f"Model accuracy {accuracy:.2f} below threshold, not saving")

              precision    recall  f1-score   support

           1       1.00      0.00      0.00     11633
           2       0.00      0.00      0.00         0

    accuracy                           0.00     11633
   macro avg       0.50      0.00      0.00     11633
weighted avg       1.00      0.00      0.00     11633

Number of valid test samples: 11633
Number of samples with missing data: 0
Model accuracy 0.00 below threshold, not saving


C:\Users\OMEN\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OMEN\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OMEN\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [45]:

try:
    test_df.to_csv("final_csv.csv", index=False)
except IOError as e:
    print(f"Error saving matches to CSV: {e}")

### test


In [46]:
train_df

invoice1                   invoice2  similarity_score
0         5514104810                 5514104892          0.668748
1         5719101009                 5719101588          0.624549
2         5514105044                 5514105198          0.587657
3        B2064/23-24                 210 /23-24          0.264735
4        B2855/23-24  SALES B - Bill # 000B2855          0.276246
...              ...                        ...               ...
9359    KAA-2223/332               KAA/2223/332          0.700428
9360     SI2223/4067            KIN/SI2223/4067          0.653983
9361   SPB-0049-0423              SPB 0049-0423          0.739689
9362  TAC-23-24-0002             TAC/23-24/0002          0.415166
9363      TPP-03-014                 TPP_03_014          0.270582

[9364 rows x 3 columns]

In [47]:
merged_df

invoice1                   invoice2  similarity_score  label
0         5514104810                 5514104892          0.668748      1
1         5719101009                 5719101588          0.624549      1
2         5514105044                 5514105198          0.587657      1
3        B2064/23-24                 210 /23-24          0.264735      1
4        B2855/23-24  SALES B - Bill # 000B2855          0.276246      1
...              ...                        ...               ...    ...
9359    KAA-2223/332               KAA/2223/332          0.700428      1
9360     SI2223/4067            KIN/SI2223/4067          0.653983      1
9361   SPB-0049-0423              SPB 0049-0423          0.739689      1
9362  TAC-23-24-0002             TAC/23-24/0002          0.415166      1
9363      TPP-03-014                 TPP_03_014          0.270582      1

[9364 rows x 4 columns]

### prediction function


In [48]:
import os

In [49]:
# class InvoiceMatchingSystem:
#     def __init__(self, model=None, vectorizer=None):
#         if model is None or vectorizer is None:
#             raise ValueError("Model and vectorizer must be provided")
#         self.model = model
#         self.vectorizer = vectorizer
#         self.feedback_buffer = []

# def process_user_feedback(matches_df, matcher):
#     """Process user feedback for invoice matches"""
#     # Input validation
#     required_cols = ['Data Set 1', 'Data Set 2']
#     if not all(col in matches_df.columns for col in required_cols):
#         raise ValueError(f"matches_df missing required columns: {required_cols}")
    
#     # Handle NaN values
#     matches_df = matches_df.dropna(subset=required_cols)
    
#     # Get potential matches
#     potential_matches = [
#         (str(row['Data Set 1']), str(row['Data Set 2'])) 
#         for _, row in matches_df.iterrows()
#     ]
    
#     try:
#         for inv1, inv2 in potential_matches:
#             # Get prediction
#             try:
#                 result = matcher.get_next_prediction(inv1, inv2)
#             except Exception as e:
#                 print(f"Error getting prediction: {e}")
#                 continue
            
#             # Display to user
#             print("\nPotential Match:")
#             print(f"Invoice 1: {inv1}")
#             print(f"Invoice 2: {inv2}")
#             print(f"Similarity Score: {result['similarity_score']:.3f}")
#             print(f"Predicted Label: {result['predicted_label']}")
            
#             # Get user feedback
#             while True:
#                 try:
#                     feedback = input("Enter feedback (0=no match, 1=partial, 2=exact, s=skip, q=quit): ")
#                     if feedback.lower() == 'q':
#                         return  # Exit function
#                     if feedback in ['0', '1', '2', 's']:
#                         break
#                     print("Invalid input. Please try again.")
#                 except KeyboardInterrupt:
#                     return  # Handle Ctrl+C gracefully
            
#             if feedback != 's':
#                 try:
#                     matcher.update_model(inv1, inv2, int(feedback))
#                 except Exception as e:
#                     print(f"Error updating model: {e}")
#                     continue
                
#     except Exception as e:
#         print(f"Error in feedback process: {e}")
#     finally:
#         # Save progress before exiting
#         matcher.save_model()

# # Usage
# try:
#     matcher = InvoiceMatchingSystem(model=model, vectorizer=vectorizer)
#     process_user_feedback(matches_df, matcher)
# except Exception as e:
#     print(f"System initialization error: {e}")

In [50]:
class InvoiceMatchingSystem:
    def __init__(self, model=None, vectorizer=None):
        self.model = model or PassiveAggressiveClassifier(max_iter=1000, random_state=42)
        self.vectorizer = vectorizer
        self.feedback_buffer = []
    
    def get_next_prediction(self, invoice1, invoice2):
        """Get prediction for a single pair"""
        X = self.vectorizer.transform([f"{invoice1} | {invoice2}"])
        # prob = self.model.predict_proba(X)[0]
        prediction = self.model.predict(X)[0]
        return {
            'Data Set 1': invoice1,
            'Data Set 2': invoice2,
            'predicted_label': prediction,
            # 'confidence': prob[prediction],
            'similarity_score': cosine_similarity(
                self.vectorizer.transform([invoice1]), 
                self.vectorizer.transform([invoice2])
            )[0][0]
        }
    
    def update_model(self, invoice1, invoice2, user_label):
        """Update model with user feedback"""
        X = self.vectorizer.transform([f"{invoice1} | {invoice2}"])
        self.model.partial_fit(X, np.array([user_label]), classes=np.array([0, 1, 2]))
        
        # Save feedback for later analysis
        self.feedback_buffer.append({
            'Data Set 1': invoice1,
            'Data Set 2': invoice2,
            'user_label': user_label,
            'timestamp': pd.Timestamp.now()
        })
        
        # Periodically save model
        if len(self.feedback_buffer) % 10 == 0:  # Save every 10 feedbacks
            self.save_model()
    
    def save_model(self):
        """Save model and feedback data"""
        joblib.dump(self.model, "invoice_match_model.joblib")
        
        # Save feedback history
        pd.DataFrame(self.feedback_buffer).to_csv(
            "feedback_history.csv", 
            mode='a', 
            header=not os.path.exists("feedback_history.csv"),
            index=False
        )

    import pandas as pd
import numpy as np
import os
from sklearn.utils.class_weight import compute_class_weight
from datetime import datetime

def process_batch_feedback(matcher, batch_size=5):
    """Process feedback in batches and update model incrementally."""
    feedback_file = "feedback_history.csv"
    potential_matches = [
        (row['Invoice1'], row['Matched_Invoice2']) 
        for _, row in matches_df.iterrows()
    ][:batch_size]
    
    feedback_batch = []
    batch_stats = {'matches': 0, 'non_matches': 0, 'partial': 0}
    
    for inv1, inv2 in potential_matches:
        result = matcher.get_next_prediction(inv1, inv2)
        
        print(f"\nPotential Match:")
        print(f"Invoice 1: {inv1}")
        print(f"Invoice 2: {inv2}")
        print(f"Similarity Score: {result['similarity_score']:.3f}")
        print(f"Predicted Label: {result['predicted_label']}")

        while True:
            feedback = input("Enter feedback (0=no match, 1=partial, 2=exact, s=skip): ")
            if feedback in ['0', '1', '2', 's']:
                break
                
        if feedback != 's':
            label = int(feedback)
            feedback_batch.append({
                'invoice1': inv1,
                'invoice2': inv2, 
                'label': label,
                'similarity_score': result['similarity_score'],
                'predicted_label': result['predicted_label'],
                'timestamp': datetime.now()
            })
            
            # Update stats
            if label == 0:
                batch_stats['non_matches'] += 1
            elif label == 1:
                batch_stats['partial'] += 1
            else:
                batch_stats['matches'] += 1

    if feedback_batch:
        # **Load Previous Feedback (Avoid Duplicates)**
        if os.path.exists(feedback_file):
            existing_feedback = pd.read_csv(feedback_file)
            feedback_df = pd.DataFrame(feedback_batch)
            feedback_df = pd.concat([existing_feedback, feedback_df], ignore_index=True).drop_duplicates()
        else:
            feedback_df = pd.DataFrame(feedback_batch)
        
        # **Save Updated Feedback**
        feedback_df.to_csv(feedback_file, index=False)

        # **Train Model on New Feedback Only**
        X_batch = [f"{f['invoice1']} | {f['invoice2']}" for _, f in feedback_df.iterrows()]
        y_batch = feedback_df["label"].values

        # **Handle Class Imbalance**
        class_weights = compute_class_weight("balanced", classes=np.array([0, 1, 2]), y=y_batch)
        class_weight_dict = {i: class_weights[i] for i in range(3)}

        # **Vectorize & Incrementally Train Model**
        X_transformed = matcher.vectorizer.transform(X_batch)
        matcher.model.partial_fit(X_transformed, y_batch, classes=np.array([0, 1, 2]), sample_weight=[class_weight_dict[y] for y in y_batch])

        matcher.save_model()

        # **Display Batch Statistics**
        print("\nBatch Statistics:")
        print(f"Exact Matches: {batch_stats['matches']}")
        print(f"Partial Matches: {batch_stats['partial']}")
        print(f"Non-matches: {batch_stats['non_matches']}")

        # **Calculate and Display Accuracy**
        print("\nModel Performance:")
        accuracy = (feedback_df['predicted_label'] == feedback_df['label']).mean()
        print(f"Accuracy: {accuracy:.2f}")

        # **Show Recent Daily Feedback Counts**
        feedback_df['timestamp'] = pd.to_datetime(feedback_df['timestamp'])
        daily_counts = feedback_df.groupby(feedback_df['timestamp'].dt.date)['label'].count()
        print("\nDaily Feedback Counts:")
        print(daily_counts.tail())

    return len(feedback_batch)




In [51]:
# Check column names
print("Available columns:", matches_df.columns.tolist())

Available columns: ['Invoice1', 'Matched_Invoice2', 'Similarity_Score']


In [52]:

# # Initialize system
# matcher = InvoiceMatchingSystem(model=model, vectorizer=vectorizer)

# # Interactive feedback loop
# def process_user_feedback():
#     # Get potential matches
#     potential_matches = [
#         (row['Invoice1'], row['Matched_Invoice2']) 
#         for _, row in matches_df.iterrows()
#     ]
    
#     for inv1, inv2 in potential_matches:
#         # Get prediction
#         result = matcher.get_next_prediction(inv1, inv2)
        
#         # Display to user
#         print(f"\nPotential Match:")
#         print(f"Invoice 1: {inv1}")
#         print(f"Invoice 2: {inv2}")
#         print(f"Similarity Score: {result['similarity_score']:.3f}")
#         print(f"Predicted Label: {result['predicted_label']}")
        
#         # Get user feedback
#         while True:
#             feedback = input("Enter feedback (0=no match, 1=partial, 2=exact, s=skip): ")
#             if feedback in ['0', '1', '2', 's']:
#                 break
        
#         if feedback != 's':
#             # Update model
#             matcher.update_model(inv1, inv2, int(feedback))

In [53]:
from typing import List, Dict, Tuple
from datetime import datetime
import pandas as pd
import numpy as np
import os

In [54]:
def process_batch_feedback(batch_size=5):
    """Process feedback in batches and maintain feedback history"""
    feedback_file = "feedback_history.csv"
    potential_matches = [
        (row['Invoice1'], row['Matched_Invoice2']) 
        for _, row in matches_df.iterrows()
    ][:batch_size]
    
    feedback_batch = []
    batch_stats = {'matches': 0, 'non_matches': 0, 'partial': 0}
    
    for inv1, inv2 in potential_matches:
        result = matcher.get_next_prediction(inv1, inv2)
        
        print(f"\nPotential Match:")
        print(f"Invoice 1: {inv1}")
        print(f"Invoice 2: {inv2}")
        print(f"Similarity Score: {result['similarity_score']:.3f}")
        print(f"Predicted Label: {result['predicted_label']}")
        
        while True:
            feedback = input("Enter feedback (0=no match, 1=partial, 2=exact, s=skip): ")
            if feedback in ['0', '1', '2', 's']:
                break
                
        if feedback != 's':
            label = int(feedback)
            feedback_batch.append({
                'invoice1': inv1,
                'invoice2': inv2, 
                'label': label,
                'similarity_score': result['similarity_score'],
                'predicted_label': result['predicted_label'],
                'timestamp': pd.Timestamp.now()
            })
            
            # Update stats
            if label == 0:
                batch_stats['non_matches'] += 1
            elif label == 1:
                batch_stats['partial'] += 1
            else:
                batch_stats['matches'] += 1
    
    # Save feedback and update model
    if feedback_batch:
        # Save feedback to CSV
        feedback_df = pd.DataFrame(feedback_batch)
        feedback_df.to_csv(feedback_file, mode='a', header=not os.path.exists(feedback_file), index=False)
        
        # Update model
        X_batch = [f"{f['invoice1']} | {f['invoice2']}" for f in feedback_batch]
        y_batch = [f['label'] for f in feedback_batch]
        X_transformed = matcher.vectorizer.transform(X_batch)
        matcher.model.partial_fit(X_transformed, np.array(y_batch), classes=np.array([0, 1, 2]))
        matcher.save_model()
        
        # Display batch statistics
        print("\nBatch Statistics:")
        print(f"Exact Matches: {batch_stats['matches']}")
        print(f"Partial Matches: {batch_stats['partial']}")
        print(f"Non-matches: {batch_stats['non_matches']}")
        
        # Display overall statistics
        if os.path.exists(feedback_file):
            all_feedback = pd.read_csv(feedback_file)
            print("\nOverall Statistics:")
            print("Label Distribution:")
            print(all_feedback['label'].value_counts())
            print("\nModel Performance:")
            accuracy = (all_feedback['predicted_label'] == all_feedback['label']).mean()
            print(f"Accuracy: {accuracy:.2f}")
            
            # Show daily feedback counts
            all_feedback['timestamp'] = pd.to_datetime(all_feedback['timestamp'])
            daily_counts = all_feedback.groupby(all_feedback['timestamp'].dt.date)['label'].count()
            print("\nDaily Feedback Counts:")
            print(daily_counts.tail())
        
    return len(feedback_batch)

# Usage
matcher = InvoiceMatchingSystem(model=model, vectorizer=vectorizer)
processed_count = process_batch_feedback(batch_size=5)


Potential Match:
Invoice 1: 5514104810
Invoice 2: 5514104892
Similarity Score: 0.669
Predicted Label: 1

Potential Match:
Invoice 1: 5719101009
Invoice 2: 5719101588
Similarity Score: 0.625
Predicted Label: 1

Potential Match:
Invoice 1: 5514105044
Invoice 2: 5514105198
Similarity Score: 0.588
Predicted Label: 1

Potential Match:
Invoice 1: B2064/23-24
Invoice 2: 210 /23-24
Similarity Score: 0.265
Predicted Label: 1

Potential Match:
Invoice 1: B2855/23-24
Invoice 2: SALES B - Bill # 000B2855
Similarity Score: 0.276
Predicted Label: 1

Batch Statistics:
Exact Matches: 2
Partial Matches: 2
Non-matches: 1

Overall Statistics:
Label Distribution:
label
1    22
2     2
0     1
Name: count, dtype: int64

Model Performance:
Accuracy: 0.88

Daily Feedback Counts:
timestamp
2025-02-06    20
2025-02-07     5
Name: label, dtype: int64


In [55]:
# Usage Example
matcher = InvoiceMatchingSystem(model=model, vectorizer=vectorizer)
processed_count = process_batch_feedback(batch_size=5) 


Potential Match:
Invoice 1: 5514104810
Invoice 2: 5514104892
Similarity Score: 0.669
Predicted Label: 1

Potential Match:
Invoice 1: 5719101009
Invoice 2: 5719101588
Similarity Score: 0.625
Predicted Label: 2

Potential Match:
Invoice 1: 5514105044
Invoice 2: 5514105198
Similarity Score: 0.588
Predicted Label: 0

Potential Match:
Invoice 1: B2064/23-24
Invoice 2: 210 /23-24
Similarity Score: 0.265
Predicted Label: 1

Potential Match:
Invoice 1: B2855/23-24
Invoice 2: SALES B - Bill # 000B2855
Similarity Score: 0.276
Predicted Label: 1

Batch Statistics:
Exact Matches: 2
Partial Matches: 2
Non-matches: 1

Overall Statistics:
Label Distribution:
label
1    24
2     4
0     2
Name: count, dtype: int64

Model Performance:
Accuracy: 0.80

Daily Feedback Counts:
timestamp
2025-02-06    20
2025-02-07    10
Name: label, dtype: int64


### Extra code (don't run)


In [56]:
import pandas as pd
import os
from datetime import datetime
from dataclasses import dataclass
from typing import List, Dict, Any

@dataclass
class FeedbackStats:
    matches: int = 0
    non_matches: int = 0
    partial: int = 0

class FeedbackProcessor:
    def __init__(self, matcher, feedback_file="feedback_history.csv"):
        self.matcher = matcher
        self.feedback_file = feedback_file
        self.stats = FeedbackStats()

    def get_potential_matches(self, matches_df: pd.DataFrame, batch_size: int) -> List[tuple]:
        return [
            (row['Invoice1'], row['Matched_Invoice2']) 
            for _, row in matches_df.iterrows()
        ][:batch_size]

    def collect_feedback(self, inv1: str, inv2: str) -> Dict[str, Any]:
        result = self.matcher.get_next_prediction(inv1, inv2)
        self._display_match(inv1, inv2, result)
        feedback = self._get_user_feedback()
        
        if feedback != 's':
            return self._create_feedback_entry(inv1, inv2, int(feedback), result)
        return None

    def _display_match(self, inv1: str, inv2: str, result: Dict[str, Any]):
        print(f"\nPotential Match:")
        print(f"Invoice 1: {inv1}")
        print(f"Invoice 2: {inv2}")
        print(f"Similarity Score: {result['similarity_score']:.3f}")
        print(f"Predicted Label: {result['predicted_label']}")

    def _get_user_feedback(self) -> str:
        while True:
            feedback = input("Enter feedback (0=no match, 1=partial, 2=exact, s=skip): ")
            if feedback in ['0', '1', '2', 's']:
                return feedback

    def _create_feedback_entry(self, inv1: str, inv2: str, label: int, result: Dict[str, Any]) -> Dict[str, Any]:
        self._update_stats(label)
        return {
            'invoice1': inv1,
            'invoice2': inv2, 
            'label': label,
            'similarity_score': result['similarity_score'],
            'predicted_label': result['predicted_label'],
            'timestamp': pd.Timestamp.now()
        }

    def _update_stats(self, label: int):
        if label == 0:
            self.stats.non_matches += 1
        elif label == 1:
            self.stats.partial += 1
        else:
            self.stats.matches += 1


In [57]:
import pandas as pd
import os
from datetime import datetime
from dataclasses import dataclass
from typing import List, Dict, Any

@dataclass
class FeedbackStats:
    matches: int = 0
    non_matches: int = 0
    partial: int = 0

class FeedbackProcessor:
    def __init__(self, matcher, feedback_file="feedback_history.csv"):
        self.matcher = matcher
        self.feedback_file = feedback_file
        self.stats = FeedbackStats()

    def get_potential_matches(self, matches_df: pd.DataFrame, batch_size: int) -> List[tuple]:
        return [
            (row['Invoice1'], row['Matched_Invoice2']) 
            for _, row in matches_df.iterrows()
        ][:batch_size]

    def collect_feedback(self, inv1: str, inv2: str) -> Dict[str, Any]:
        result = self.matcher.get_next_prediction(inv1, inv2)
        self._display_match(inv1, inv2, result)
        feedback = self._get_user_feedback()
        
        if feedback != 's':
            return self._create_feedback_entry(inv1, inv2, int(feedback), result)
        return None

    def _display_match(self, inv1: str, inv2: str, result: Dict[str, Any]):
        print(f"\nPotential Match:")
        print(f"Invoice 1: {inv1}")
        print(f"Invoice 2: {inv2}")
        print(f"Similarity Score: {result['similarity_score']:.3f}")
        print(f"Predicted Label: {result['predicted_label']}")

    def _get_user_feedback(self) -> str:
        while True:
            feedback = input("Enter feedback (0=no match, 1=partial, 2=exact, s=skip): ")
            if feedback in ['0', '1', '2', 's']:
                return feedback

    def _create_feedback_entry(self, inv1: str, inv2: str, label: int, result: Dict[str, Any]) -> Dict[str, Any]:
        self._update_stats(label)
        return {
            'invoice1': inv1,
            'invoice2': inv2, 
            'label': label,
            'similarity_score': result['similarity_score'],
            'predicted_label': result['predicted_label'],
            'timestamp': pd.Timestamp.now()
        }

    def _update_stats(self, label: int):
        if label == 0:
            self.stats.non_matches += 1
        elif label == 1:
            self.stats.partial += 1
        else:
            self.stats.matches += 1

class FeedbackAnalyzer:
    def __init__(self, feedback_file: str):
        self.feedback_file = feedback_file

    def display_batch_stats(self, stats: FeedbackStats):
        print("\nBatch Statistics:")
        print(f"Exact Matches: {stats.matches}")
        print(f"Partial Matches: {stats.partial}")
        print(f"Non-matches: {stats.non_matches}")

    # def display_overall_stats(self):
    #     if not os.path.exists(self.feedback_file):
    #         return

    #     all_feedback = pd.read_csv(self.feedback_file)
    #     print("\nOverall Statistics:")
    #     self._display_label_distribution(all_feedback)
    #     self._display_model_performance(all_feedback)
    #     self._display_daily_counts(all_feedback)

    # def _display_label_distribution(self, df: pd.DataFrame):
    #     print("Label Distribution:")
    #     print(df['label'].value_counts())

    # def _display_model_performance(self, df: pd.DataFrame):
    #     accuracy = (df['predicted_label'] == df['label']).mean()
    #     print("\nModel Performance:")
    #     print(f"Accuracy: {accuracy:.2f}")

    # def _display_daily_counts(self, df: pd.DataFrame):
    #     df['timestamp'] = pd.to_datetime(df['timestamp'])
    #     daily_counts = df.groupby(df['timestamp'].dt.date)['label'].count()
    #     print("\nDaily Feedback Counts:")
    #     print(daily_counts.tail())

def process_batch_feedback(matches_df: pd.DataFrame, matcher, batch_size: int = 5) -> int:
    processor = FeedbackProcessor(matcher)
    analyzer = FeedbackAnalyzer(processor.feedback_file)
    feedback_batch = []

    potential_matches = processor.get_potential_matches(matches_df, batch_size)
    
    for inv1, inv2 in potential_matches:
        feedback_entry = processor.collect_feedback(inv1, inv2)
        if feedback_entry:
            feedback_batch.append(feedback_entry)

    if feedback_batch:
        save_feedback_and_update_model(feedback_batch, processor.feedback_file, matcher)
        analyzer.display_batch_stats(processor.stats)
        # analyzer.display_overall_stats()

    return len(feedback_batch)

def save_feedback_and_update_model(feedback_batch: List[Dict[str, Any]], feedback_file: str, matcher):
    feedback_df = pd.DataFrame(feedback_batch)
    feedback_df.to_csv(feedback_file, mode='a', header=not os.path.exists(feedback_file), index=False)
    print("asdsadsa")
    print(feedback_batch)
    print(feedback_df.columns.tolist())
    X_batch = [f"{f['invoice1']} | {f['invoice2']}" for f in feedback_batch]
    print("Xbatch: ", X_batch)
    y_batch = [f['label'] for f in feedback_batch]
    print("ybatch: ", y_batch)
    X_transformed = matcher.vectorizer.transform(X_batch)
    matcher.model.partial_fit(X_transformed, np.array(y_batch), classes=np.array([0, 1, 2]))
    matcher.save_model()

In [58]:
matcher = InvoiceMatchingSystem(model=model, vectorizer=vectorizer)
processed_count = process_batch_feedback(matches_df, matcher, batch_size=5)


Potential Match:
Invoice 1: 5514104810
Invoice 2: 5514104892
Similarity Score: 0.669
Predicted Label: 1

Potential Match:
Invoice 1: 5719101009
Invoice 2: 5719101588
Similarity Score: 0.625
Predicted Label: 2

Potential Match:
Invoice 1: 5514105044
Invoice 2: 5514105198
Similarity Score: 0.588
Predicted Label: 2

Potential Match:
Invoice 1: B2064/23-24
Invoice 2: 210 /23-24
Similarity Score: 0.265
Predicted Label: 1

Potential Match:
Invoice 1: B2855/23-24
Invoice 2: SALES B - Bill # 000B2855
Similarity Score: 0.276
Predicted Label: 2
asdsadsa
[{'invoice1': '5514104810', 'invoice2': '5514104892', 'label': 0, 'similarity_score': 0.6687475161745501, 'predicted_label': 1, 'timestamp': Timestamp('2025-02-07 23:38:58.870315')}, {'invoice1': '5719101009', 'invoice2': '5719101588', 'label': 2, 'similarity_score': 0.6245490351869111, 'predicted_label': 2, 'timestamp': Timestamp('2025-02-07 23:39:02.386723')}, {'invoice1': '5514105044', 'invoice2': '5514105198', 'label': 1, 'similarity_score':

In [59]:
from typing import List, Dict, Tuple
from datetime import datetime
import pandas as pd
import numpy as np
import os

        ### I can replace the feedback_df entries in train_df and then retrain the whole model or else I can or I can incrementally train on only those entries


In [60]:

def process_batch_feedback_new(
    invoice_pairs: List[Tuple[str, str]], 
    feedback_labels: List[int], 
    batch_size: int = 5
) -> Dict:
    """Process feedback in batches and maintain feedback history
    
    Args:
        invoice_pairs: List of tuples containing (invoice1, invoice2) pairs
        feedback_labels: List of user feedback labels (0=no match, 1=partial, 2=exact)
        batch_size: Size of batch for processing feedback (default: 5)
        
    Returns:
        Dict containing batch statistics and model performance metrics
    """
    try:
        feedback_file = "feedback_history.csv"
        # if not 0 < batch_size <= 100:
        #     raise ValueError("Batch size must be between 1 and 100")
            
        feedback_batch = []
        batch_stats = {'matches': 0, 'non_matches': 0, 'partial': 0}
        
        # Process feedback batch
        for (inv1, inv2), label in zip(invoice_pairs[:batch_size], feedback_labels[:batch_size]):
            result = matcher.get_next_prediction([inv1], [inv2])[0]
            
            feedback_batch.append({
                'invoice1': inv1,
                'invoice2': inv2,
                'label': label,
                'similarity_score': result['similarity_score'],
                'predicted_label': result['predicted_label'],
                'timestamp': pd.Timestamp.now()
            })
            
            # Update stats
            if label == 0:
                batch_stats['non_matches'] += 1
            elif label == 1:
                batch_stats['partial'] += 1
            else:
                batch_stats['matches'] += 1

        # Save and update model if batch not empty
        if feedback_batch:
            feedback_df = pd.DataFrame(feedback_batch)
            feedback_df.to_csv(feedback_file, mode='a', 
                             header=not os.path.exists(feedback_file), 
                             index=False)
            
            # Update model
            X_batch = [f"{f['invoice1']} | {f['invoice2']}" for f in feedback_batch]
            y_batch = [f['label'] for f in feedback_batch]
            X_transformed = matcher.vectorizer.transform(X_batch)
            matcher.model.partial_fit(X_transformed, np.array(y_batch), 
                                    classes=np.array([0, 1, 2]))
            
            # Save updated model
            matcher.save_model()
            
            # Calculate performance metrics
            metrics = calculate_performance_metrics(feedback_file)
            return {
                'batch_stats': batch_stats,
                'metrics': metrics,
                'processed_count': len(feedback_batch)
            }
            
        return {'error': 'No feedback to process'}
        
    except Exception as e:
        return {'error': str(e)}

def calculate_performance_metrics(feedback_file: str) -> Dict:
    """Calculate model performance metrics from feedback history"""
    if not os.path.exists(feedback_file):
        return {}
        
    all_feedback = pd.read_csv(feedback_file)
    all_feedback['timestamp'] = pd.to_datetime(all_feedback['timestamp'])
    
    return {
        'accuracy': (all_feedback['predicted_label'] == all_feedback['label']).mean(),
        'label_distribution': all_feedback['label'].value_counts().to_dict(),
        'daily_counts': all_feedback.groupby(all_feedback['timestamp'].dt.date)['label'].count().to_dict()
    }

In [61]:
feedback = pd.read_csv("feedback_history.csv")
feedback

invoice1                   invoice2  label  similarity_score  \
0    5514104810                 5514104892      1          0.668748   
1    5719101009                 5719101588      1          0.624549   
2    5514105044                 5514105198      1          0.587657   
3   B2064/23-24                 210 /23-24      1          0.264735   
4   B2855/23-24  SALES B - Bill # 000B2855      1          0.276246   
5    5514104810                 5514104892      1          0.668748   
6    5719101009                 5719101588      1          0.624549   
7    5514105044                 5514105198      1          0.587657   
8   B2064/23-24                 210 /23-24      1          0.264735   
9   B2855/23-24  SALES B - Bill # 000B2855      1          0.276246   
10   5514104810                 5514104892      1          0.668748   
11   5719101009                 5719101588      1          0.624549   
12   5514105044                 5514105198      1          0.587657   
13  B2064/23-24                 210 /23-24      1          0.264735   
14  B2855/23-24  SALES B - Bill # 000B2855      1          0.276246   
15   5514104810                 5514104892      1          0.668748   
16   5719101009                 5719101588      1          0.624549   
17   5514105044                 5514105198      1          0.587657   
18  B2064/23-24                 210 /23-24      1          0.264735   
19  B2855/23-24  SALES B - Bill # 000B2855      1          0.276246   
20   5514104810                 5514104892      1          0.668748   
21   5719101009                 5719101588      2          0.624549   
22   5514105044                 5514105198      0          0.587657   
23  B2064/23-24                 210 /23-24      1          0.264735   
24  B2855/23-24  SALES B - Bill # 000B2855      2          0.276246   
25   5514104810                 5514104892      1          0.668748   
26   5719101009                 5719101588      0          0.624549   
27   5514105044                 5514105198      2          0.587657   
28  B2064/23-24                 210 /23-24      1          0.264735   
29  B2855/23-24  SALES B - Bill # 000B2855      2          0.276246   
30   5514104810                 5514104892      0          0.668748   
31   5719101009                 5719101588      2          0.624549   
32   5514105044                 5514105198      1          0.587657   
33  B2064/23-24                 210 /23-24      1          0.264735   
34  B2855/23-24  SALES B - Bill # 000B2855      2          0.276246   

    predicted_label                   timestamp  
0                 1  2025-02-06 10:26:28.881312  
1                 1  2025-02-06 10:26:30.093400  
2                 1  2025-02-06 10:26:31.132595  
3                 1  2025-02-06 10:26:32.154217  
4                 1  2025-02-06 10:26:33.511894  
5                 1  2025-02-06 10:26:46.125609  
6                 1  2025-02-06 10:26:47.646138  
7                 1  2025-02-06 10:26:49.277089  
8                 1  2025-02-06 10:26:50.627161  
9                 1  2025-02-06 10:26:52.565978  
10                1  2025-02-06 10:29:31.789373  
11                1  2025-02-06 10:29:33.893847  
12                1  2025-02-06 10:29:34.968282  
13                1  2025-02-06 10:29:36.021893  
14                1  2025-02-06 10:29:37.354212  
15                1  2025-02-06 10:29:39.214497  
16                1  2025-02-06 10:29:40.227064  
17                1  2025-02-06 10:29:41.048518  
18                1  2025-02-06 10:29:41.599616  
19                1  2025-02-06 10:29:42.889142  
20                1  2025-02-07 22:56:24.173206  
21                1  2025-02-07 22:56:28.809651  
22                1  2025-02-07 22:56:31.530992  
23                1  2025-02-07 22:56:32.905791  
24                1  2025-02-07 22:56:34.099644  
25                1  2025-02-07 22:56:37.029383  
26                2  2025-02-07 23:38:50.415100  
27                0  2025-02-07 23:38:52.690442  


In [62]:

matcher = InvoiceMatchingSystem(model=model, vectorizer=vectorizer)
batch_size = len(feedback)
processed_count = process_batch_feedback_new(matches_df, matcher, batch_size=batch_size)



In [63]:
try:
    matcher = InvoiceMatchingSystem(model=model, vectorizer=vectorizer)
    batch_size = len(feedback)
    if batch_size == 0:
        raise ValueError("Empty feedback batch")
        
    results = process_batch_feedback_new(matches_df, matcher, batch_size=batch_size)
    if results is None:
        raise ValueError("No results returned from processing")
        
    print(f"Processing complete. Results: {results}")
    print(results)
    
except Exception as e:
    print(f"Error in feedback processing: {str(e)}")
    print(None)

Processing complete. Results: {'error': "'InvoiceMatchingSystem' object is not subscriptable"}
{'error': "'InvoiceMatchingSystem' object is not subscriptable"}


In [64]:

# Print results
print("Feedback Processing Results:")
print(f"Processed Count: {processed_count.get('processed_count', 0)}")
print(f"Batch Stats: {processed_count.get('batch_stats', {})}")

if 'error' in processed_count:
    print(f"Error occurred: {processed_count['error']}")

Feedback Processing Results:
Processed Count: 0
Batch Stats: {}
Error occurred: 'InvoiceMatchingSystem' object is not subscriptable
